In [2]:
import sys
sys.path.append('../utils')
sys.path.append('../core')
sys.path.append('../')
import numpy as np
import scipy
import tqdm
import utils
import geometry
from core import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import glob

from functools import reduce
from math import sqrt

In [75]:
def largest_prime_factor(n): 
    largest_prime = -1
    i = 2
    while i * i <= n: 
        while n % i == 0: 
            largest_prime = i 
            n = n // i 
        i = i + 1
    if n > 1: 
        largest_prime = n 
    return largest_prime 

# Computation in kwave utilizes a fourier-space calculation, therefore computational grid sizes with low factor numbers are ideal.
def calc_matrix_size(grid_size, voxel_size):
    matrix_size = []
    raw_matrix_size = np.array(grid_size) / np.array(voxel_size)
    for dim in range(3):
        lpfs = []
        for i in range(int(raw_matrix_size[dim]), int(raw_matrix_size[dim] * 1.5)):
            lpfs.append(largest_prime_factor(i))
        matrix_size.append(raw_matrix_size[dim] + np.argmin(lpfs))
    return matrix_size

In [81]:
calc_matrix_size((270,381,900), (1,1,1))

[288.0, 512.0, 1024.0]

In [ ]:
def compute_vertex_transforms(self,):
    get size of computational grid in global coordinates
    get the positions of each vertex of the computational grid - make sure to account for PML
    get the transforms (in this case, a simple translation) associated with each vertex, put into a list, and append (optional, also possible to just save the coordinates)
    
    new_grid_size = (np.array(self.matrix_size) - np.array(self.PML_size)) * np.array(self.voxel_dims)
    
    
        return 0